# Conditional GMM
Created by: Daniel L. Marino (marinodl@vcu.edu)

Description: gaussian mixture model implementation on tensorflow, the objective is a conditional probabilistic distribution. 

This file serves as a visual test for the gmm extension modules

In [1]:
import math
import numpy as np
import collections
from time import time
import tensorflow as tf
from twodlearn.tf_lib.datasets.generic import Datasets
from twodlearn.tf_lib.GMM import *

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## 1. Create dataset

In [2]:

def gmm_sampling(mu, sigma, w, n_samples=1):
    # generates random vectors, sampled from a gausian mixture model
    #  
    #     - mu: 3d tensor containing the means for the gaussians.
    #           the "depth" dimention (3rd) is used to index the
    #           gaussians.    [ kernel_id, dim]
    #     - sigma: 3d tensor containing the covariance matrix of the
    #              gaussians. [ kernel_id, dim] for diagonal matrices
    #     - w: vector in form of a 3d tensor containing the weights
    #          for each one of the gaussians, they have to sum one. 
    #          [kernel_id]
    n_kernels = mu.shape[0]
    n_dim = mu.shape[1]
    
    # random sample the kernel from which the output samples are going
    # to be drawn
    kernel_id= np.argmax(np.random.multinomial(1, w, size=[n_samples]), axis=1 )
    
    out = np.zeros([n_samples, n_dim])
    for i in range(n_samples):
        out[i,:]= np.random.multivariate_normal(mu[kernel_id[i],:], np.diag(sigma[kernel_id[i],:])) # if diagonal
    
    return out;
    
    

In [3]:
n_samples = 1000000

sigma_r = 1.0*np.exp(-0.005*np.linspace(0, 1000, n_samples))
w_r = [1]  # has to sum to one

aux_x= np.linspace(0, 6, n_samples)
aux_y= np.linspace(0, 6, n_samples)

for i in range(n_samples):
    aux_y[i] = np.random.normal(np.sin( aux_x[i] ), sigma_r[i])

# build the dataset
data = Datasets(aux_x, aux_y)
data.normalize()
print('Data shape: ', data.train.x.shape)

# plot
#plt.plot(data.train.x, data.train.y, 'o')

Data shape:  (1000000,)


In [4]:
print(np.sin(aux_x[2]))

1.20000119997e-05


## 2. Model definition

In [5]:
#config = tf.ConfigProto( device_count = {'GPU': 0} )
#sess = tf.InteractiveSession(config = config)
sess = tf.InteractiveSession()

In [6]:
n_inputs = 1
n_outputs = 1
n_hidden= [4]
n_kernels = 1


gmm_model = GmmMlpModel( n_inputs, n_outputs, n_hidden, n_kernels, 
                             afunction= tf.sigmoid, diagonal= True, method='tdl')

train_model= gmm_model.setup(n_samples)
    
# Optimizer.
optimizer = tf.train.AdamOptimizer(0.002).minimize(train_model.loss) #0.001

In [7]:
print('mu shape:', train_model.mu.get_shape())
print('sigma shape:', train_model.sigma.get_shape())
print('w shape:', train_model.w.get_shape())

print('out shape:', train_model.out.get_shape())

print('loss shape:', train_model.loss.get_shape())


mu shape: (1000000, 1, 1)
sigma shape: (1000000, 1, 1)
w shape: (1000000, 1)
out shape: <unknown>
loss shape: <unknown>


## 3. Training

In [8]:
num_steps = 10000 #1000
n_logging = 100
n_test_logg = 10

tf.initialize_all_variables().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0

t0 = time()
for step in range(num_steps):   
    
    _, l = sess.run([optimizer, train_model.loss],feed_dict={train_model.inputs : np.expand_dims(data.train.x,1),
                                                             train_model.labels : np.expand_dims(data.train.y,1)})
    mean_loss += l    
    
    
    if step%n_logging == 0:                
        print(step, ' | loss:', mean_loss/n_logging)
        mean_loss= 0
        
t1 = time()
print('function takes: ', (t1-t0))

Initialized
0  | loss: 0.0127263879776
100  | loss: 1.20739375353
200  | loss: 1.10066717982
300  | loss: 0.850288107395
400  | loss: 0.651929777265
500  | loss: 0.598655427098
600  | loss: 0.560335657597
700  | loss: 0.523335894346
800  | loss: 0.485066242814
900  | loss: 0.449415409565
1000  | loss: 0.411708776057
1100  | loss: 0.360139915347
1200  | loss: 0.29437020272
1300  | loss: 0.223771533519
1400  | loss: 0.154679082185
1500  | loss: 0.0965109656006
1600  | loss: 0.0549155367166
1700  | loss: 0.0236063866457
1800  | loss: -0.0145678763546
1900  | loss: -0.0819727937132
2000  | loss: -0.191443995982
2100  | loss: -0.339002245367
2200  | loss: -0.51268802613
2300  | loss: -0.693941006064
2400  | loss: -0.842288567424
2500  | loss: -0.932336855531
2600  | loss: -0.981260842085
2700  | loss: -1.00948995292
2800  | loss: -1.02590925455
2900  | loss: -1.03553029656
3000  | loss: -1.04157194972
3100  | loss: -1.04605837345
3200  | loss: -1.04909473538
3300  | loss: -1.05194606662
340

## 4. Plot

In [9]:
# model for testing
n_test= 100;
test_model = gmm_model.setup(n_test)


In [10]:
x_plot = np.linspace(-2, 2, n_test)

[mu_out, sigma_out, w_out,] = sess.run([ test_model.mu, test_model.sigma, test_model.w ],
                                         feed_dict= {test_model.inputs : np.expand_dims(x_plot, 1)})
#print('mu:', mu_out, 'sigma:', sigma_out, 'w:', w_out)

print(np.squeeze(mu_out).shape)
print(x_plot.shape)

# plot
#plt.plot(x_plot, np.squeeze(mu_out), 'o')

#plt.plot(x_plot, np.squeeze(mu_out) + np.squeeze(np.sqrt(sigma_out)), 'ro')
#plt.plot(x_plot, np.squeeze(mu_out) - np.squeeze(np.sqrt(sigma_out)), 'ro')


(100,)
(100,)
